<a href="https://colab.research.google.com/github/Vishy-A/DeepLearning/blob/main/HW5/4106HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import requests

In [3]:
text = "Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction lays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology."


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
chars = sorted(list(set(text)))
ind_to_char = {i : ch for i, ch in enumerate(chars)}
char_to_ind = {ch : i for i, ch in enumerate(chars)}

In [6]:
maxlen = 20
x = []
y = []
for i in range(len(text) - maxlen):
  sequence = text[i : i + maxlen]
  label = text[i + maxlen]
  x.append([char_to_ind[ch] for ch in sequence])
  y.append(char_to_ind[label])

In [7]:
x = np.array(x)
y = np.array(y)

In [8]:
xtrain, xval, ytrain, yval = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
xtrain = torch.tensor(xtrain, dtype=torch.long)
xval = torch.tensor(xval, dtype=torch.long)
ytrain = torch.tensor(ytrain, dtype=torch.long)
yval = torch.tensor(yval, dtype=torch.long)

In [10]:
class CharTransformer(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers, nhead):
    super(CharTransformer, self).__init__()
    self.embedding = nn.Embedding(input_size, hidden_size)
    encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
    self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    embedded = self.embedding(x)
    trans_output = self.transformer_encoder(embedded)
    output = self.fc(trans_output[:, -1, :])
    return output

In [11]:
hidden_size = 128
n_layers = 3
nhead = 2
lr = .005
epochs = 100

model = CharTransformer(len(chars), hidden_size, len(chars), n_layers, nhead)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(xtrain)
  loss = criterion(outputs, ytrain)
  loss.backward()
  optimizer.step()

  model.eval()
  with torch.no_grad():
    valoutput = model(xval)
    valloss = criterion(valoutput, yval)
    _, valpred = torch.max(valoutput, dim=1)
    valacc = (valpred == yval).float().mean()

  if(epoch+1) % 10 == 0:
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Val Loss: {valloss.item():.4f}, Val Acc: {valacc.item():.4f}")

In [ ]:
def predict_next_char(model, char_to_ind, ind_to_char, initial_str):
  model.eval()
  with torch.no_grad():
    initial_ip = torch.tensor([char_to_ind[c] for c in initial_str[-maxlen:]], dtype = torch.long).unsqueeze(0)
    prediction = model(initial_ip)
    predicted_ind = torch.argmax(prediction, dim=1).item()
    return ind_to_char[predicted_ind]

In [ ]:
test_str = "This is a simple example to demonstrate how to predict the next cha"
predicted_char = predict_next_char(model, char_to_ind, ind_to_char, test_str)
print(f"Predicted character: {predicted_char}")